In [1]:
import pyswarms as ps
import numpy as np
import pandas as pd
from pyswarms.utils.search import RandomSearch
from pyswarms.utils.functions import single_obj as fx
import matplotlib.pyplot as plt
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename= 'dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','Smpl','Avg','Cmplx','UUCW','TCF','ECF','UCP','Real_Effort']
cols = pd.DataFrame(col)

cols.insert(9,"Desired_UUCW", 0 , True)
cols

,UAW,Smpl,Avg,Cmplx,UUCW,TCF,ECF,UCP,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [3]:
for index, row in cols.iterrows():
    UAW = cols.loc[index,"UAW"]
    UUCW = cols.loc[index,"UUCW"]
    TCF = cols.loc[index,"TCF"]
    ECF = cols.loc[index,"ECF"]
    Real_E = cols.loc[index,"Real_Effort"]    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    cols.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)
    
print(cols)

   UAW Smpl Avg Cmplx UUCW   TCF   ECF     UCP Real_Effort  Desired_UUCW
1    9    6  10    15  355  0.81  0.84   398.5        7970           576
2    8    4  20    15  445  0.99  0.99   398.1        7962           398
3    9    1   5    20  355  1.03   0.8  396.75        7935           472
4    8    5  10    15  350   0.9  0.91  390.25        7805           468
5    8    1  10    16  345   0.9  0.91   387.9        7758           465
..  ..  ...  ..   ...  ...   ...   ...     ...         ...           ...
67  18    2  18    18  460  1.05  0.95     290        5800           272
68  17    1  23    22  565  1.03   0.8  289.55        5791           334
69  19    5  30    10  475  0.95  0.92   289.1        5782           311
70   6    5  15     5  250     1  0.92   288.9        5778           308
71  18    5  18    17  460  0.85  0.89  288.75        5775           363

[71 rows x 10 columns]


In [4]:
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)
# Perform optimization
best_cost, best_pos = optimizer.optimize(fx.sphere, iters=100)

2021-04-07 20:11:18,969 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=7.45e-8
2021-04-07 20:11:19,190 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 7.446441080188751e-08, best pos: [0.00022623 0.0001526 ]


In [5]:
gen = 20
target = 576 
tcf = 0.81
ecf = 0.84
uaw = 9
best_smpl = 0
best_avg = 0
best_cmplx = 0
best_gap = 100
for i in range(gen):
  optimizer1 = ps.single.GlobalBestPSO(n_particles=100, dimensions=2, options=options)
  optimizer2 = ps.single.GlobalBestPSO(n_particles=100, dimensions=2, options=options)
  optimizer3 = ps.single.GlobalBestPSO(n_particles=100, dimensions=2, options=options)

  best_cost1, best_pos1 = optimizer1.optimize(fx.sphere, iters=100)
  best_cost2, best_pos2 = optimizer2.optimize(fx.sphere, iters=100)
  best_cost3, best_pos3 = optimizer3.optimize(fx.sphere, iters=100)

  smpl = 5*best_cost1
  avg = 10*best_cost2
  cmplx = 15*best_cost3

  uucw = smpl + avg + cmplx

  uucp = uaw + uucw

  ucp = uucp * tcf * ecf

  gap = np.abs(ucp-target)

  print()
  print('generasi ke', i)
  print("ucp", ucp)
  print("smpl", smpl)
  print("avg", avg)
  print("cmplx", cmplx)

  if gap < best_gap:
    best_smpl = smpl
    best_avg = avg
    best_cmplx = cmplx

2021-04-07 20:11:24,344 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.24e-8
2021-04-07 20:11:24,523 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.2394120054937396e-08, best pos: [9.71771889e-05 5.43204750e-05]
2021-04-07 20:11:24,525 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.16e-8
2021-04-07 20:11:24,673 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.1647811683005201e-08, best pos: [-4.25947417e-05 -9.91640039e-05]
2021-04-07 20:11:24,675 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|████████████████████████


generasi ke 0
ucp 6.123600156087344
smpl 6.197060027468699e-08
avg 1.1647811683005202e-07
cmplx 5.0956549660101236e-08


pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████|100/100, best_cost=8.35e-10
2021-04-07 20:11:25,094 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8.350928314436578e-10, best pos: [ 2.84793626e-05 -4.90089133e-06]
2021-04-07 20:11:25,095 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=3.64e-9
2021-04-07 20:11:25,301 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.6356569362848492e-09, best pos: [ 3.91666697e-05 -4.58435265e-05]
2021-04-07 20:11:25,305 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████|100/100, best_cost=3.6e-9
2021-04-07 20:11:25,556 - pyswarms.single.global_best - INFO 


generasi ke 1
ucp 6.123600064318559
smpl 4.175464157218289e-09
avg 3.635656936284849e-08
cmplx 5.3998476703659185e-08


pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████|100/100, best_cost=1.2e-8
2021-04-07 20:11:25,848 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.199869547846932e-08, best pos: [ 5.72432158e-05 -9.33911651e-05]
2021-04-07 20:11:25,850 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=3.17e-9
2021-04-07 20:11:26,064 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.172828541577131e-09, best pos: [-4.76940736e-05  2.99683814e-05]
2021-04-07 20:11:26,065 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.44e-8
2021-04-07 20:11:26,275 - pyswarms.single.global_best - INFO -

...


generasi ke 17
ucp 6.123600234497546
smpl 1.0713450616390385e-07
avg 2.3428254521157011e-07
cmplx 3.229547834513175e-09


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=4.18e-9
2021-04-07 20:11:36,486 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.179106419722834e-09, best pos: [4.91046151e-05 4.20457274e-05]
2021-04-07 20:11:36,488 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=4.57e-9
2021-04-07 20:11:36,684 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 4.566805484730753e-09, best pos: [ 3.94499500e-05 -5.48680866e-05]
2021-04-07 20:11:36,686 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=4.64e-9
2021-04-07 20:11:36,883 - pyswarms.single.global_best - INFO - O


generasi ke 18
ucp 6.123600092685554
smpl 2.089553209861417e-08
avg 4.5668054847307525e-08
cmplx 6.965856734446549e-08


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.03e-8
2021-04-07 20:11:37,170 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.0309235319188837e-08, best pos: [1.01464525e-04 3.76637556e-06]
2021-04-07 20:11:37,172 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=9.28e-9
2021-04-07 20:11:37,373 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 9.28030678207898e-09, best pos: [-2.88083739e-05  9.19259723e-05]
2021-04-07 20:11:37,377 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████|100/100, best_cost=2.4e-9
2021-04-07 20:11:37,583 - pyswarms.single.global_best - INFO - O


generasi ke 19
ucp 6.1236001227464865
smpl 5.1546176595944185e-08
avg 9.28030678207898e-08
cmplx 3.6054173616658076e-08


In [6]:
print(best_smpl)
print(best_avg)
print(best_cmplx)

0
0
0
